### Interface Qui Englobe  identify_speake, verify_speaker and recognize_language

In [1]:
from tkinter import messagebox
import tkinter as tk
from tkinter import filedialog
import pygame
import pickle
import os
from tkinter import ttk
import scipy.io.wavfile as wav
from python_speech_features import mfcc
import numpy as np 

pygame 2.4.0 (SDL 2.26.4, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class AudioPlayer:
    def __init__(self, root):
        pygame.mixer.init()
        self.root = root
        self.file = None
        self.is_paused = False
       
        self.root.title("Audio Player")
        self.root.geometry("500x400")  #

        # Create a golden frame
        golden_frame = tk.Frame(self.root, bg="gold2")
        golden_frame.pack(pady=10, padx=20, fill=tk.BOTH, expand=True)

        # frame dyal the title
        title_frame = tk.Frame(golden_frame, bg="gold2")
        title_frame.pack(pady=10)
       # the title label
        title_label = tk.Label(title_frame, text="Traitement de Parole", font=("Helvetica", 24, "bold"), bg="gold2", fg="white")
        title_label.pack()

        # Create a frame for buttons
        button_frame = tk.Frame(golden_frame, bg="gold2")
        button_frame.pack(pady=10)

        open_button = tk.Button(button_frame, text="Open", command=self.open_file, bg="gold3", fg="white", font=("Helvetica", 12, "bold"))
        open_button.grid(row=0, column=0, padx=5)

        play_button = tk.Button(button_frame, text="Play", command=self.play, state=tk.DISABLED, bg="gold3", fg="white", font=("Helvetica", 12, "bold"))
        play_button.grid(row=0, column=1, padx=5)

        pause_button = tk.Button(button_frame, text="Pause/Unpause", command=self.pause, state=tk.DISABLED, bg="gold3", fg="white", font=("Helvetica", 12, "bold"))
        pause_button.grid(row=0, column=2, padx=5)

        identify_button = tk.Button(button_frame, text="Identify Speaker", command=self.identify_speaker, state=tk.DISABLED, bg="gold3", fg="white", font=("Helvetica", 12, "bold"))
        identify_button.grid(row=0, column=3, padx=5)

        gmm_frame = tk.Frame(golden_frame, bg="gold2")
        gmm_frame.pack(pady=10)

        self.gmm_label = tk.Label(gmm_frame, text="Select GMM model:", font=("Helvetica", 12), bg="gold2")
        self.gmm_label.grid(row=0, column=0, padx=5)

        self.gmm_var = tk.StringVar(root)
        self.gmm_dropdown = ttk.OptionMenu(gmm_frame, self.gmm_var, "128", "256", "512", "1024")
        self.gmm_dropdown.config(style="G.TMenubutton")
        self.gmm_dropdown.grid(row=0, column=1, padx=5)

        style = ttk.Style()
        style.configure("G.TMenubutton", background="gold3", foreground="white", font=("Helvetica", 12, "bold"))

    
        audio_info_frame = tk.Frame(golden_frame, bg="gold2")
        audio_info_frame.pack(pady=10)

        self.audio_name_label = tk.Label(audio_info_frame, text="Audio Name: ", font=("Helvetica", 12), bg="gold2", fg="white")
        self.audio_name_label.pack(side=tk.LEFT)

        self.audio_duration_label = tk.Label(audio_info_frame, text="Duration: ", font=("Helvetica", 12), bg="gold2", fg="white")
        self.audio_duration_label.pack(side=tk.LEFT)

        # Create a frame for result display
        result_frame = tk.Frame(golden_frame, bg="gold2")
        result_frame.pack(pady=10)

        self.result_label = tk.Label(result_frame, text="", font=("Helvetica", 12, "bold"), bg="gold2")
        self.result_label.pack()

        verify_label = tk.Label(result_frame, text="Verify Speaker:", font=("Helvetica", 12), bg="gold2")
        verify_label.pack(pady=5)

        self.label_entry = tk.Entry(result_frame, width=20)
        self.label_entry.pack()

        verify_button = tk.Button(result_frame, text="Verify", command=self.verify_speaker, state=tk.DISABLED, bg="gold3", fg="white", font=("Helvetica", 12, "bold"))
        verify_button.pack(pady=5)

        self.verified_labels_label = tk.Label(result_frame, text="Verified Labels:", font=("Helvetica", 10, "italic"), bg="gold2")
        self.verified_labels_label.pack()
        
        
        language_button = tk.Button(button_frame, text="Recognize Language", command=self.recognize_language, state=tk.DISABLED, bg="gold3", fg="white", font=("Helvetica", 12, "bold"))
        language_button.grid(row=0, column=4, padx=5)


        self.buttons = {
            "open_button": open_button,
            "play_button": play_button,
            "pause_button": pause_button,
            "identify_button": identify_button,
            "verify_button": verify_button,
            "language_button": language_button
        }

        
    def open_file(self):
        self.file = filedialog.askopenfilename(filetypes=[("Audio Files", "*.wav")])
        if not self.file:
            return
        self.buttons["play_button"].config(state=tk.NORMAL)
        self.buttons["pause_button"].config(state=tk.NORMAL)
        self.buttons["identify_button"].config(state=tk.NORMAL)
        self.buttons["verify_button"].config(state=tk.NORMAL)
        self.buttons["language_button"].config(state=tk.NORMAL)

        # Load and display audio information
        audio_name = os.path.basename(self.file)
        audio_duration = self.get_audio_duration(self.file)

        self.audio_name_label.config(text=f"Audio Name: {audio_name}")
        self.audio_duration_label.config(text=f"Duration: {audio_duration}")

    def play(self):
        pygame.mixer.music.load(self.file)
        pygame.mixer.music.play()

    def pause(self):
        if self.is_paused:
            pygame.mixer.music.unpause()
            self.is_paused = False
        else:
            pygame.mixer.music.pause()
            self.is_paused = True

    def load_gmm_model(self, file_path):
        with open(file_path, 'rb') as f:
            gmm_model = pickle.load(f)
        return gmm_model

    def load_mfcc_features(self, file_path):
        with open(file_path, 'r') as f:
            lines = f.readlines()
        mfcc_features = [list(map(float, line.strip().split())) for line in lines]
        return mfcc_features

    def identify_speaker(self):
        # Assume that the selected .wav file has corresponding .mfcc file in the specific directory
        filename = os.path.basename(self.file)
        filename_without_ext = os.path.splitext(filename)[0]

        if filename.startswith('F'):
            # Load MFCC features for F
            mfcc_file_path = os.path.join(r'C:\Users\aorus\Desktop\s8\TP\Projet2\MFCC\MFCC_testF', filename_without_ext + ".mfcc")
        elif filename.startswith('H'):
            # Load MFCC features for H
            mfcc_file_path = os.path.join(r'C:\Users\aorus\Desktop\s8\TP\Projet2\MFCC\MFCC_testH', filename_without_ext + ".mfcc")
        else:
            messagebox.showerror("Error", "Invalid audio file format")
            return

        mfcc_features = self.load_mfcc_features(mfcc_file_path)

        models_path = r'C:\Users\aorus\Desktop\s8\TP\Projet2\GMM'
        selected_gmm = self.gmm_var.get()  # Get the selected GMM model from the dropdown menu

        # Get GMM files for F and H based on the selected GMM size
        gmm_files = [f for f in os.listdir(models_path) if f.endswith(f".{selected_gmm}.gmm")]

        gmms = []
        labels = []

        # Load GMM models for F and H
        for gmm_file in gmm_files:
            file_path = os.path.join(models_path, gmm_file)
            gmm = self.load_gmm_model(file_path)
            label = gmm_file[:-4]  # Remove the '.gmm' extension to get the label
            gmms.append(gmm)
            labels.append(label)

            # Calculate log likelihoods for F and H
            log_likelihoods = [gmm.score(mfcc_features) for gmm in gmms]

            # Get the label of the GMM with maximum likelihood
            max_likelihood_index = log_likelihoods.index(max(log_likelihoods))
            best_model_label = labels[max_likelihood_index]
            self.identified_speaker = best_model_label

            # Create a mapping between labels and names
            labels_to_names = {
                'F1': 'Ikram',
                'F2': 'Chaimae',
                'F3': 'Khadija',
                'F4': 'Khadija',
                'F5': 'Firdawse',
                'F6': 'Kaoutar',
                'F7': 'Salima',
                'F8': 'Oumaima',
                'F9': 'Imane'
            }

            # Find the corresponding name
            speaker_name = labels_to_names.get(best_model_label)

            # Check if the speaker_name exists in the dictionary
            if speaker_name:
                self.result_label.config(text=f"The speaker is more likely to be: {best_model_label} {speaker_name}")
            else:
                self.result_label.config(text=f"The speaker is more likely to be: {best_model_label}")
     
   

    def verify_speaker(self):
        if not self.file:
            return

        # Assume that the selected .wav file has corresponding .mfcc file in the specific directory
        filename = os.path.basename(self.file)
        filename_without_ext = os.path.splitext(filename)[0]
        speaker_label = self.label_entry.get()  # Get the speaker label from the entry field

        if filename.startswith('F'):
            # Load MFCC features for F
            mfcc_file_path = os.path.join(r'C:\Users\aorus\Desktop\s8\TP\Projet2\MFCC\MFCC_testF', filename_without_ext + ".mfcc")
        elif filename.startswith('H'):
            # Load MFCC features for H
            mfcc_file_path = os.path.join(r'C:\Users\aorus\Desktop\s8\TP\Projet2\MFCC\MFCC_testH', filename_without_ext + ".mfcc")
        else:
            messagebox.showerror("Error", "Invalid audio file format")
            return

        mfcc_features = self.load_mfcc_features(mfcc_file_path)

        # Load GMM models for verification
        models_path = r'C:\Users\aorus\Desktop\s8\TP\Projet2\GMM'
        selected_gmm = self.gmm_var.get()  # Get the selected GMM model from the dropdown menu

        # Get GMM files based on the selected GMM size
        gmm_files = [f for f in os.listdir(models_path) if f.endswith(f".{selected_gmm}.gmm")]

        gmms = []
        labels = []

        # Load GMM models
        for gmm_file in gmm_files:
            file_path = os.path.join(models_path, gmm_file)
            gmm = self.load_gmm_model(file_path)
            label = gmm_file.split(".")[0]  # Extract the label from the file name
            gmms.append(gmm)
            labels.append(label)

        # Calculate log likelihoods for all speakers
        log_likelihoods = [gmm.score(mfcc_features) for gmm in gmms]

        # Set the threshold for log likelihood score
        threshold = -70.25147

        # Get the labels with log likelihood scores above the threshold
        above_threshold_labels = [label for label, score in zip(labels, log_likelihoods) if score >= threshold]

        # Check if the provided speaker label belongs to the above threshold labels
        if speaker_label in above_threshold_labels:
            verification_result = "Verified: Client"
        else:
            verification_result = "Not Verified: Imposter"

        # Update the label text with the verification result
        self.result_label.config(text=verification_result)
        
        # Display the verified labels with log likelihood scores above the threshold
        verified_labels = ", ".join(above_threshold_labels)
        verified_labels_text = f"Verified Labels: {verified_labels}"
        self.verified_labels_label.config(text=verified_labels_text)

    def get_audio_duration(self, file_path):
        audio = pygame.mixer.Sound(file_path)
        duration = audio.get_length()
        minutes = int(duration // 60)
        seconds = int(duration % 60)
        return f"{minutes:02d}:{seconds:02d}"
        
        
    def recognize_language(self):
        if not self.file:
            return

        base_path = r"C:\Users\aorus\Desktop\s8\TP\Projet"

        with open(f'{base_path}\gmm_ru.gmm', 'rb') as f:
            gmm_ru= pickle.load(f)
        with open(f'{base_path}\gmm_fr.gmm', 'rb') as f:
            gmm_fr = pickle.load(f)
        with open(f'{base_path}\gmm_en.gmm', 'rb') as f:
            gmm_en = pickle.load(f)
        with open(f'{base_path}\gmm_ar.gmm', 'rb') as f:
            gmm_ar = pickle.load(f)
        with open(f'{base_path}\gmm_sp.gmm', 'rb') as f:
            gmm_sp = pickle.load(f)

        (rate, signal) = wav.read(self.file)
        mfcc_feature = mfcc(signal, rate)

        best_language, score = self.predict(mfcc_feature, gmm_ru, gmm_fr, gmm_en, gmm_ar, gmm_sp)

        messagebox.showinfo("Language Recognition", f"The speaker's language is likely to be: {best_language}")

    def predict(self, test, gmm_ru, gmm_fr, gmm_en, gmm_ar, gmm_sp):
        langues = ["russe","french","anglais","arabe","espagnol","allemand"]
        means = []
        log_likelihood1 = []
        log_likelihood2 = []
        log_likelihood3 = []
        log_likelihood4 = []
        log_likelihood5 = []

        for i in range(len(test)):
            log_likelihood1.append(gmm_ru.score_samples(test[i].reshape(1, -1)))
            log_likelihood2.append(gmm_fr.score_samples(test[i].reshape(1, -1)))
            log_likelihood3.append(gmm_en.score_samples(test[i].reshape(1, -1)))
            log_likelihood4.append(gmm_ar.score_samples(test[i].reshape(1, -1)))
            log_likelihood5.append(gmm_sp.score_samples(test[i].reshape(1, -1)))

        means.append(np.mean( log_likelihood1))
        means.append(np.mean(log_likelihood2))
        means.append(np.mean(log_likelihood3))
        means.append(np.mean(log_likelihood4))
        means.append(np.mean(log_likelihood5))

        index = np.argmax(means)
        return langues[index], max(means)
    
def main():
    root = tk.Tk()
    player = AudioPlayer(root)
    root.mainloop()

if __name__ == "__main__":
    main()

In [ ]:
labels